In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv
/kaggle/input/nlp-getting-started/sample_submission.csv


In [2]:
!pip install transformers==2.11.0 --quiet
!pip install simpletransformers==0.41.0 --quiet
#!pip install nlpaug numpy matplotlib python-dotenv --quiet
!pip install pyspellchecker --quiet

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [3]:
import random
import torch

In [4]:
from simpletransformers.classification import ClassificationModel
import pandas as pd

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [5]:
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [6]:
import warnings
warnings.simplefilter('ignore')
from sklearn.model_selection import KFold, StratifiedKFold
from scipy.special import softmax
import sklearn
from sklearn.metrics import log_loss, f1_score
import numpy as np

In [7]:
import random
import numpy as np
import torch
def seed_all(seed_value):
    random.seed(seed_value) # Python
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
        torch.backends.cudnn.benchmark = False

seed_all(79)

In [8]:
train = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")
print("Shape of train data : ",train.shape)
print("Shape of test data : ",test.shape)

Shape of train data :  (7613, 5)
Shape of test data :  (3263, 4)


In [9]:
# Remove the mislabelled tweets
#incorrect_labels_df = train.groupby(['text']).nunique().sort_values(by='target', ascending=False)
#incorrect_labels_df = incorrect_labels_df[incorrect_labels_df['target'] > 1]
#incorrect_texts = incorrect_labels_df.index.tolist()
#train = train[~train.text.isin(incorrect_texts)]

In [10]:

# Add the keyword column to the text column
train['keyword'].fillna('', inplace=True)
train['final_text'] = train['keyword'] + ' ' + train['text'] 
test['keyword'].fillna('', inplace=True)
test['final_text'] = test['keyword'] + ' ' + test['text'] 

In [11]:
train.head()

,id,keyword,location,text,target,final_text
0,1,,NaN,Our Deeds are the Reason of this #earthquake M...,1,Our Deeds are the Reason of this #earthquake ...
1,4,,NaN,Forest fire near La Ronge Sask. Canada,1,Forest fire near La Ronge Sask. Canada
2,5,,NaN,All residents asked to 'shelter in place' are ...,1,All residents asked to 'shelter in place' are...
3,6,,NaN,"13,000 people receive #wildfires evacuation or...",1,"13,000 people receive #wildfires evacuation o..."
4,7,,NaN,Just got sent this photo from Ruby #Alaska as ...,1,Just got sent this photo from Ruby #Alaska as...


In [12]:
train=train.drop(['id'],axis=1)
train=train.drop(['keyword'],axis=1)
train=train.drop(['text'],axis=1)
train=train.drop(['location'],axis=1)
train.head()

,target,final_text
0,1,Our Deeds are the Reason of this #earthquake ...
1,1,Forest fire near La Ronge Sask. Canada
2,1,All residents asked to 'shelter in place' are...
3,1,"13,000 people receive #wildfires evacuation o..."
4,1,Just got sent this photo from Ruby #Alaska as...


In [13]:
first_col = ['final_text']
last_cols = [col for col in train.columns if col not in first_col]

train = train[first_col+last_cols]
train.head()

,final_text,target
0,Our Deeds are the Reason of this #earthquake ...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are...,1
3,"13,000 people receive #wildfires evacuation o...",1
4,Just got sent this photo from Ruby #Alaska as...,1


In [14]:
final=pd.DataFrame()
final['id']=test['id']
final.head()

,id
0,0
1,2
2,3
3,9
4,11


In [15]:
test=test.drop(['id'],axis=1)
test=test.drop(['keyword'],axis=1)
test=test.drop(['text'],axis=1)
test=test.drop(['location'],axis=1)
test['label']=0
test.head()

,final_text,label
0,Just happened a terrible car crash,0
1,"Heard about #earthquake is different cities, ...",0
2,"there is a forest fire at spot pond, geese ar...",0
3,Apocalypse lighting. #Spokane #wildfires,0
4,Typhoon Soudelor kills 28 in China and Taiwan,0


In [16]:
first_col = ['final_text']
last_cols = [col for col in test.columns if col not in first_col]

test = test[first_col+last_cols]
test.head()

,final_text,label
0,Just happened a terrible car crash,0
1,"Heard about #earthquake is different cities, ...",0
2,"there is a forest fire at spot pond, geese ar...",0
3,Apocalypse lighting. #Spokane #wildfires,0
4,Typhoon Soudelor kills 28 in China and Taiwan,0


In [17]:
train['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [18]:
test.head()

,final_text,label
0,Just happened a terrible car crash,0
1,"Heard about #earthquake is different cities, ...",0
2,"there is a forest fire at spot pond, geese ar...",0
3,Apocalypse lighting. #Spokane #wildfires,0
4,Typhoon Soudelor kills 28 in China and Taiwan,0


In [19]:
4313/3245

1.3291217257318952

In [20]:
train = train.reindex(np.random.permutation(train.index))
train= train.reset_index(drop=True)
train.head()

,final_text,target
0,derailment #ModiMinistry Madhya Pradesh Train ...,1
1,annihilation @KimKardashian can you please sig...,0
2,suicide%20bomb wo Pic of 16yr old PKK suicide ...,1
3,exploded @OKgooner hahaha great song. 'Spent 1...,0
4,weapons PM Abe pledged to make every effort to...,1


In [21]:
from sklearn.model_selection import KFold, StratifiedKFold
from scipy.special import softmax

In [22]:
f1=sklearn.metrics.f1_score

In [23]:
# model configuration
model_args = { 
    "save_eval_checkpoints": False,
     "save_model_every_epoch": False,
      'reprocess_input_data': True,
       'overwrite_output_dir': True,
       'manual_seed': 79,
       "silent": True, 
    'num_train_epochs': 2,
    'learning_rate': 2e-5,
    'fp16': False,
    'max_seq_length': 64,


}

#'max_seq_length': 128,
#            'evaluate_during_training': True,
 #   'evaluate_during_training_steps': 1000,
  #    'use_early_stopping': True,
   # 'early_stopping_patience': 3,

In [24]:
%%time
torch.cuda.empty_cache()
kf = StratifiedKFold(n_splits=15, shuffle=True, random_state=79)
err=[]
y_pred_tot=[]
for train_index, test_index in kf.split(train, train['target']):
  train1_trn, train1_val = train.iloc[train_index], train.iloc[test_index]
  model_rb = ClassificationModel('roberta', 'roberta-base', weight=[1, 1.329], args=model_args)
  model_rb.train_model(train1_trn, eval_df=train1_val)
  result, model_outputs, _ = model_rb.eval_model(train1_val, f1=sklearn.metrics.f1_score, acc=sklearn.metrics.accuracy_score)
  print(f"f1 score:{result['f1']}", f"Accuracy:{result['acc']}")    
  err.append((result['f1']))
  predictions, _ = model_rb.predict(test['final_text'])
  y_pred_tot.append(predictions)
print("Mean f1 score: ",np.mean(err))


f1 score:0.8551724137931035 Accuracy:0.8759842519685039
f1 score:0.800982800982801 Accuracy:0.8405511811023622
f1 score:0.8066825775656324 Accuracy:0.8405511811023622
f1 score:0.8167053364269143 Accuracy:0.844488188976378
f1 score:0.8271028037383178 Accuracy:0.8543307086614174
f1 score:0.7971360381861575 Accuracy:0.8326771653543307
f1 score:0.7972350230414746 Accuracy:0.8267716535433071
f1 score:0.7999999999999998 Accuracy:0.8326771653543307
f1 score:0.794392523364486 Accuracy:0.8264299802761341
f1 score:0.7630331753554502 Accuracy:0.8027613412228797
f1 score:0.832116788321168 Accuracy:0.863905325443787
f1 score:0.7964205816554809 Accuracy:0.8205128205128205
f1 score:0.8151658767772513 Accuracy:0.8461538461538461
f1 score:0.8047058823529412 Accuracy:0.8362919132149902
f1 score:0.7926267281105991 Accuracy:0.8224852071005917
Mean f1 score:  0.806631903311452
CPU times: user 44min 7s, sys: 1min, total: 45min 7s
Wall time: 48min 42s


In [25]:
to_submit =np.mean(y_pred_tot,0)

In [26]:
to_submit

array([1., 1., 1., ..., 1., 1., 1.])

In [27]:
#predictions, raw_outputs = model_rb.predict(test['final_text'])
final['target']=to_submit
final['target'] = final['target'].apply(lambda x: 1 if x>0.5 else 0)
final.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1


In [28]:
final.to_csv('model_robert_base_lr2e5_ep2_skf15_.csv',index=False)